In [ ]:
PLAYER = 0 # 0 -> white, 1 -> black
GAME_NAME = "test1"
pgn = """1. d4 {[%clk 0:29:59]} 1... d5 {[%clk 0:29:57.1]} 2. Nf3 {[%clk 0:29:55.4]} 2...
        Nc6 {[%clk 0:29:46.9]} 3. Bf4 {[%clk 0:29:53.3]} 3... e6 {[%clk 0:29:38.2]} 4.
        e3 {[%clk 0:29:52]} 4... Bb4+ {[%clk 0:29:32.6]} 5. c3 {[%clk 0:29:49.9]} 5...
        Ba5 {[%clk 0:29:28.5]} 6. Nbd2 {[%clk 0:29:48]} 6... Nf6 {[%clk 0:29:16.3]} 7.
        Bb5 {[%clk 0:29:10.3]} 7... Ng4 {[%clk 0:29:05.1]} 8. O-O {[%clk 0:28:53.1]}
        8... a6 {[%clk 0:28:52.7]} 9. Ba4 {[%clk 0:28:48.2]} 9... O-O {[%clk 0:28:46.6]}
        10. h3 {[%clk 0:28:44]} 10... Nf6 {[%clk 0:28:33.2]} 11. Bxc6 {[%clk 0:28:29.1]}
        11... bxc6 {[%clk 0:28:30.1]} 12. Ne5 {[%clk 0:28:04.8]} 12... Ne4 {[%clk
        0:28:05.1]} 13. Nxe4 {[%clk 0:27:54.2]} 13... dxe4 {[%clk 0:28:03.4]} 14. Qg4
        {[%clk 0:27:51.5]} 14... g6 {[%clk 0:27:04.2]} 15. Nxc6 {[%clk 0:27:17.1]} 15...
        Qd5 {[%clk 0:26:39.3]} 16. Ne7+ {[%clk 0:27:12.5]} 16... Kg7 {[%clk 0:26:26.9]}
        17. Nxd5 {[%clk 0:27:02]} 17... exd5 {[%clk 0:26:25.2]} 18. Be5+ {[%clk
        0:26:52.6]} 18... Kg8 {[%clk 0:26:13.4]} 19. Qh4 {[%clk 0:26:37.8]} 19... f6
        {[%clk 0:25:44.7]} 20. Bxf6 {[%clk 0:26:25]} 20... Bf5 {[%clk 0:25:21.1]} 21.
        Rac1 {[%clk 0:25:20.3]} 21... Rab8 {[%clk 0:25:13.8]} 22. b4 {[%clk 0:25:06.2]}
        22... Bb6 {[%clk 0:24:52.9]} 23. c4 {[%clk 0:24:45.2]} 23... a5 {[%clk
        0:24:45.8]} 24. bxa5 {[%clk 0:24:29.7]} 24... Bxa5 {[%clk 0:24:44.3]} 25. cxd5
        {[%clk 0:24:28.4]} 25... Rb5 {[%clk 0:24:28.7]} 26. Be5 {[%clk 0:24:02.5]} 26...
        Rxd5 {[%clk 0:24:06.2]} 27. Bxc7 {[%clk 0:23:58.9]} 27... Bxc7 {[%clk
        0:24:03.6]} 28. Rxc7 {[%clk 0:23:57.5]} 28... Rf7 {[%clk 0:23:55.1]} 29. Rxf7
        {[%clk 0:23:42.3]} 29... Kxf7 {[%clk 0:23:53.2]} 30. Rb1 {[%clk 0:23:41.2]}
        30... Ra5 {[%clk 0:23:44.2]} 31. Rb7+ {[%clk 0:23:37.3]} 31... Ke8 {[%clk
        0:23:34.3]} 32. Qe7# {[%clk 0:23:32.7]} 1-0"""

In [ ]:
import chess.pgn
import chess.svg
import chess.engine
import copy
import io
from IPython.display import SVG, display
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import os

def import_pgn(pgn: str) -> chess.pgn.Game:
    pgn_io = io.StringIO(pgn)
    game = chess.pgn.read_game(pgn_io)
    return game

engine = chess.engine.SimpleEngine.popen_uci("/home/narcis/Stockfish/src/stockfish")

trp_green = "rgba(0, 255, 0, 0.5)"
trp_blue = "rgba(0, 0, 255, 0.5)"
trp_light_green = "rgba(144, 238, 144, 0.5)"
trp_orange = "rgba(255, 165, 0, 0.5)"
trp_yellow = "rgba(255, 255, 0, 0.5)"
trp_red = "rgba(255, 0, 0, 0.5)"
trp_pink = "rgba(255, 192, 203, 0.5)"
trp_cyan = "rgba(0, 255, 255, 0.5)"
trp_brown = "rgba(165, 42, 42, 0.5)"
trp_magenta = "rgba(255, 0, 255, 0.5)"

In [ ]:
def get_score(info: chess.engine.InfoDict) -> int:
    # Get the evaluation in centipawns
    if info["score"].is_mate():
        mate_score = abs(info["score"].relative.moves)
        if mate_score == 0:
            mate_score = 1
        # Assign a high positive or negative value if it's a mate situation
        if info["score"].turn:
            score = 4000/mate_score
        else:
            score = -4000/mate_score
    else:
        score = info["score"].relative.score()
    return -score

def analyse_movement(board: chess.Board, move: chess.Move):
    # Analyse the performed movement
    board.push(move)
    info = engine.analyse(board, chess.engine.Limit(time=0.1))
    board.pop()
    real_score = (move, get_score(info))

    # Analyse all the possible movements
    scores = []
    for p_move in board.legal_moves:
        board.push(p_move)
        info = engine.analyse(board, chess.engine.Limit(time=0.1))
        board.pop()
        scores.append((p_move, get_score(info)))
    
    print("Real movement:", real_score, " Best movement:", max(scores, key=lambda x: x[1]))
    return real_score, scores

def display_board(board: chess.Board, arrows:list, player:int, filename:str) -> None:
    svg = chess.svg.board(board, arrows=arrows, size=400, flipped=(player==1))
    display(SVG(svg))
    # Save the SVG to a file
    if filename is not None:
        with open(filename, 'w') as f:
            f.write(svg)

def check_movement(real_score, scores):
    color = trp_red
    difference = real_score[1] - max(scores, key=lambda x: x[1])[1]
    if real_score[0] == max(scores, key=lambda x: x[1])[0]:
        print("Best movement!")
        color = trp_blue
    elif difference > -30:
        print("Excellent Movement:", difference)
        color = trp_green
    elif difference > -75:
        print("Good Movement:", difference)
        color = trp_light_green
    elif difference > -150:
        print("Blunder:", difference) 
        color = trp_yellow
    elif difference > -250:
        print("Error:", difference) 
        color = trp_orange
    else:
        print("Big Mistake:", difference)
        color = trp_red

    # get all the movements with a similar score (up to 50 points) from the best one
    similar_scores = [x for x in scores if abs(x[1] - max(scores, key=lambda x: x[1])[1]) < 50]
    # sort similar scores by the best movement
    similar_scores.sort(key=lambda x: x[1], reverse=True)
    # if real_score[0] in list of tuples similar_scores remove it. 
    for i, score in enumerate(similar_scores):
        if real_score[0] == score[0]:
            similar_scores.pop(i)
            break
    return color, similar_scores

def analyse_game(game: chess.pgn.Game, board: chess.Board) -> list:
    # Analyze all board movements
    all_scores = []
    all_differences = []
    if not os.path.exists(GAME_NAME):
        os.makedirs(GAME_NAME)
    for i, move in enumerate(game.mainline_moves()):
        arrows = []
        if i >= 0 and i % 2 == PLAYER:
            real_score, scores = analyse_movement(board, move)
            all_differences.append(real_score[1] - max(scores, key=lambda x: x[1])[1])
            color, best_movements = check_movement(real_score, scores)
            arrows.append(chess.svg.Arrow(move.from_square, move.to_square, color=color))
            display_board(board, arrows, PLAYER, filename="{}/move_{:03d}.svg".format(GAME_NAME, i))
            for k, m in enumerate(best_movements):
                if k == 0: 
                    c=trp_magenta
                else:
                    c=trp_pink
                arrows.append(chess.svg.Arrow(m[0].from_square, m[0].to_square, color=c))
            display_board(board, arrows, PLAYER, filename="{}/move_sol{:03d}.svg".format(GAME_NAME, i))
            all_scores.append(real_score[1])  
            
        board.push(move)
    return all_scores, all_differences

In [ ]:
def create_game_plots(all_scores, all_differences):
    # Create a new figure
    plt.figure(figsize=(10, 6))

    # Plot the scores
    clipped_scores = np.clip(all_scores, -1000, 1000)
    plt.plot(clipped_scores, color='blue', linestyle='-')
    plt.axis([0, len(clipped_scores), 0, max(clipped_scores)])

    # Add labels and title
    plt.xlabel('Move number')
    plt.ylabel('Score')
    plt.title('Game Analysis')

    # Add a grid
    plt.grid(True)

    # Show the plot
    plt.savefig("{}/scores.png".format(GAME_NAME), dpi=300, transparent=True)
    plt.clf()

    # Plot the differences
    a = np.clip(all_differences, -1000, 0)
    plt.plot(a, color='blue', linestyle='-')
    
    # Add labels and title
    plt.xlabel('Move number')
    plt.ylabel('Score Difference')
    plt.title('Game Analysis')

    # Add a grid
    plt.grid(True)

    # Show the plot
    plt.savefig("{}/differences.png".format(GAME_NAME), dpi=300, transparent=True)


In [ ]:
# Load game from PGN
game = import_pgn(pgn)

# Initialize board onject
board = game.board()

# Analyze all board movements
all_scores, all_differences = analyse_game(game, board)

# Plot the scores
create_game_plots(all_scores, all_differences)